In [ ]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

def download_data():
    print('start')
    startTm = datetime.datetime.now()

    startDate = (datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y%m%d')
    endDate = (datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y%m%d')

    readPath = '\\\\192.168.10.28\\equityTradeLogs'
    dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
    dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
    dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

    date = dateLs[0]
    assert(date == (datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y%m%d'))

    readPath = '\\\\192.168.10.28\\equityTradeLogs'
    orderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%date))

    for col in ['clockAtArrival', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        orderLog[col] = orderLog[col].fillna(0)
        orderLog[col] = orderLog[col].astype('int64')

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)
    orderLog = orderLog[orderLog["secid"] >= 1000000]

    targetStock = orderLog['secid'].unique()
    targetStock = np.array([int(str(i)[1:]) for i in targetStock])
    targetStockSZ = sorted(targetStock[targetStock < 600000])
    targetStockSH = sorted(targetStock[targetStock >= 600000])

    readPath = '\\\\192.168.10.34\\random_backup\\Kevin_zhenyu\\rawData'
    mdOrderLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdOrderLog***.csv'))[-1]
    mdTradeLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdTradeLog***.csv'))[-1]

    mdOrderLog = pd.read_csv(mdOrderLogPath)
    mdOrderLog = mdOrderLog[mdOrderLog['SecurityID'].isin(targetStockSZ)]
    mdOrderLog['OrderType'] = mdOrderLog['OrderType'].astype(str)
    mdOrderLog = mdOrderLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'Side',
                         'OrderType', 'Price', 'OrderQty']]

    mdTradeLog = pd.read_csv(mdTradeLogPath, encoding='utf-8')
    mdTradeLog['ExecType'] = mdTradeLog['ExecType'].astype(str)
    mdTradeLog = mdTradeLog[mdTradeLog['SecurityID'].isin(targetStockSZ)]
    mdTradeLog['volumeThisUpdate'] = np.where(mdTradeLog['ExecType'] == 'F', mdTradeLog['TradeQty'], 0)
    mdTradeLog['cum_volume'] = mdTradeLog.groupby(['SecurityID'])['volumeThisUpdate'].cumsum()
    mdTradeLog = mdTradeLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'cum_volume',
                         'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']]

    mdMsgData = pd.concat([mdOrderLog, mdTradeLog], sort=False)
    del mdOrderLog
    del mdTradeLog

    mdMsgData = mdMsgData.sort_values(by=['sequenceNo']).reset_index(drop=True)

    mdMsgData["agg_trade"] = np.where((mdMsgData["ApplSeqNum"] == mdMsgData["BidApplSeqNum"] + 1) & (mdMsgData["ExecType"] == "F"), 1, np.where(
    (mdMsgData["ApplSeqNum"] == mdMsgData["OfferApplSeqNum"] + 1) & (mdMsgData["ExecType"] == "F"), 1, 0))
    mdMsgData["agg"] = mdMsgData.groupby(["SecurityID"])["agg_trade"].shift(-1)
    mdMsgData["orderNum"] = np.where(mdMsgData["ExecType"].isnull(), 1, 0)
    mdMsgData["cumorderNum"] = mdMsgData.groupby("SecurityID")["orderNum"].cumsum()
    mdMsgData["cumorderNum2"] = np.nan
    mdMsgData.loc[mdMsgData["agg"]==1, "cumorderNum2"] = mdMsgData.loc[mdMsgData["agg"]==1, "cumorderNum"]
    mdMsgData["cumorderNum2"] = mdMsgData.groupby("SecurityID")["cumorderNum2"].ffill()
    mdMsgData.loc[mdMsgData["cumorderNum2"] == mdMsgData["cumorderNum"], "cum_volume"] = mdMsgData[mdMsgData["cumorderNum2"] == mdMsgData["cumorderNum"]]\
    .groupby(["SecurityID", "cumorderNum"])["cum_volume"].transform("max")

    mdMsgData = mdMsgData.sort_values(by=['sequenceNo']).reset_index(drop=True)

    mdMsgData['cum_volume'] = mdMsgData.groupby(['SecurityID'])['cum_volume'].ffill()
    mdMsgData['cum_volume'] = mdMsgData.groupby(['SecurityID'])['cum_volume'].backfill()
    mdMsgData['ExecType'] = mdMsgData['ExecType'].fillna('2')
    mdMsgData['TradeQty'] = mdMsgData['TradeQty'].fillna(0)

    saveCols = ['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'cum_volume', 'ApplSeqNum', 
            'Side', 'OrderType', 'Price', 'OrderQty', 'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney',
            'BidApplSeqNum', 'OfferApplSeqNum', "agg"]
    mdMsgData = mdMsgData[saveCols]
    savePath = 'L:\\orderLog\\mdData'
    mdMsgData.to_pickle(os.path.join(savePath, 'mdLog_msg_%s.pkl'%date))
    del mdMsgData

    print(datetime.datetime.now() - startTm)
    
    
    
    
    
    
    

    pd.set_option('max_rows', 200)
    pd.set_option('max_columns', 200)

    perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

    startTm = datetime.datetime.now()

    startDate = (datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y%m%d')
    endDate = (datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y%m%d')

    readPath = '\\\\192.168.10.28\\equityTradeLogs'
    dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
    dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
    dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

    thisDate = dateLs[0]
    assert(thisDate == (datetime.datetime.today() - datetime.timedelta(1)).strftime('%Y%m%d'))
    
    readPath = '\\\\192.168.10.28\\equityTradeLogs'
    rawOrderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%thisDate))
    for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
                'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        rawOrderLog[col] = rawOrderLog[col].fillna(0)
        rawOrderLog[col] = rawOrderLog[col].astype('int64')   
    rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep=False)]\
    .groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep='first')

    display('There are ticks with orderDirection 0')
    display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
                'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

    assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    try:
        assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    except:
        print('There are orders with all things same except sdd')
        print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)])
        assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
    try:
        assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
    except:
        print('There are orders with same internalId but different orderId other than accCode 8856 case')
        print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

    r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
    r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
    r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize']).grouper.group_info[0]
    r1 = r1.sort_values(by=['test', 'clockAtArrival'])
    r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
    r1['vai'] = r1.groupby('test')['vai'].ffill()
    r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
    r2 = r2.sort_values(by=['test', 'clockAtArrival'])
    r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
    r2['vai'] = r2.groupby('test')['vai'].ffill()
    assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    try:
        assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    except:
        a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
        print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
    rawOrderLog = pd.concat([r1, r2])
    del r1
    del r2  

    rawOrderLog['clock'] = rawOrderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    rawOrderLog["broker"] = np.where(rawOrderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), rawOrderLog['accCode'] // 10000, rawOrderLog['accCode'] // 100)
    rawOrderLog['colo_broker'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['broker'].astype('str')
    rawOrderLog['colo_account'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['accCode'].astype('str')
    rawOrderLog['order'] = rawOrderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai']).grouper.group_info[0]
    rawOrderLog['group'] = rawOrderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
    rawOrderLog['startClock'] = rawOrderLog.groupby(['order'])['clockAtArrival'].transform('first')
    rawOrderLog['duration'] = rawOrderLog['clockAtArrival'] - rawOrderLog['startClock']
    rawOrderLog['orderPrice'] = rawOrderLog['orderPrice'].apply(lambda x: round(x, 2))
    rawOrderLog['tradePrice'] = rawOrderLog['tradePrice'].apply(lambda x: round(x, 2))
    rawOrderLog['orderDirection1'] = np.where(rawOrderLog["orderDirection"] == -2, -1, np.where(
        rawOrderLog["orderDirection"] == 2, 1, rawOrderLog["orderDirection"]))
    orderLog = rawOrderLog.copy()


    ### Assertion 1:  make sure same direction in same date, secid, vai
    print('=======================================================================================')
    print('1. same date, secid, vai: same direction')
    orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
    if len(orderLog[orderLog['directNum'] != 1]) > 0:
        print('opposite direction for same date, same secid, same vai')
        display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)][['date', 'accCode', 'secid', 'vai', 'orderDirection', 'order']])
        orderLog = orderLog[orderLog['directNum'] == 1]

    assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

    ## Assertion 2:  make sure each account, secid, vai only has one insertion
    print('=======================================================================================')
    print('2. same date, secid, vai, accCode: one insertion')
    a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
    if len(a[a > 1]) > 0:
        print('more than one insertion at same time')
        a = a[a>1].reset_index()
        display(a)
        orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

    orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                                 np.where(orderLog['mse'] == 100, 1, 0), np.nan)
    orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

    placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
    print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


    ### Assertion 3:  check IPO stocks selling status
    print('=======================================================================================')
    print('3. IPO stocks selling (ars = 301, 302)')
    if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
        kk = orderLog[orderLog['ars'].isin([301, 302])]
        print(kk)
        try:
            assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
            print('we only sell, never buy')
        except:
            print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk[kk['orderDirection1'] == 1])
        kk1 = kk[kk['updateType'] == 0]
        kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
        kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
        kk1['diff'] = kk1['diff'].fillna(0)
        try:
            assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
            print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
        except:
            print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk1[kk1['diff'] < 10e6])
        kk2 = kk[(kk['updateType'] == 1)]
        try:
            assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
            print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
        except:
            print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk2[kk2['duration'] < 3e6])


    ### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
    print('=======================================================================================')
    print('4. updateType 7 orders')
    if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
        assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['order'].nunique().max() < 20)
        assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['order'].nunique().sum() < 100)

    ### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
    print('=======================================================================================')
    print('5. updateType 6 orders')
    k1 = orderLog[orderLog['updateType'] == 6].groupby('accCode')['order'].nunique().reset_index()
    k2 = orderLog.groupby('accCode')['order'].nunique().reset_index()
    k = pd.merge(k1, k2, on='accCode', how='left')
    k['prob'] = k['order_x']/k['order_y']
    try:
        assert(sum(k['prob'] >= 0.05) == 0)
    except:
        print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(k[k['prob'] >= 0.05])

    ### Assertion 6: check CYB orders, make sure CYB stocks total absOrderSize < 30w
    print('=======================================================================================')
    print('6. CYB stocks total order size < 30w')
    try:
        assert(orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)]['absOrderSize'].max() <= 300000)
    except:
        print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')


    ### Assertion 7:  make sure there is no unexpected updateType 
    print('=======================================================================================')
    print('7. unexpected updateType')
    def getTuple(x):
        return tuple(i for i in x)

    checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
    checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                         np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                                   (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                                   (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                         np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

    orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
    orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

    ### Assertion 8:  make sure status==0 got all traded
    print('=======================================================================================')
    print('8. status == 0: all traded')
    a = orderLog[orderLog['status'] == 0]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    print('in total trade, any fill != total cases')
    display(a[a['filled'] != a['total']])
    if a[a['filled'] != a['total']].shape[0] > 0:
        removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### Assertion 9:  make sure status==1 got partial traded
    print('=======================================================================================')
    print('9. status == 1: partial traded')
    a = orderLog[orderLog['status'] == 1]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
    display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
    if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
        removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### Assertion 10: make sure no cancellation within 1 sec
    print('=======================================================================================')
    print('10. no cancellation within 1 sec')
    a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
    print('any cancellation within 1 sec')
    display(a)
    if a.shape[0] > 0:
        removeOrderLs = a['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


    ### Assertion 11: make sure no order has shares > 80w or notional > 800w
    print('=======================================================================================')
    print('11. Orders with size > 80w or notional > 800w')
    orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
    if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
        print('some order quantity are > 80w')
        print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
        display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                             'orderNtl', 'orderDirection', 'clock', 'order']])

    if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
        print('some order ntl are > 800w')
        print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
        display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                          'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                          "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

    removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
    orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
    orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

    orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
    orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
    if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
        display(orderLog[orderLog['mrsb90'] == '-'])
    orderLog = orderLog[orderLog['mrsb90'] != '-']
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
    if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
        display(orderLog[orderLog['aaa'] == '-'])
    orderLog = orderLog[orderLog['aaa'] != '-']
    orderLog['aaa'] = orderLog['aaa'].astype(float)
    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']  

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
    orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
    orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
    orderLog = orderLog[orderLog['secid'] >= 2000000].reset_index(drop=True)

    # 1. market orders
    readPath = 'L:\\orderLog\\mdData'
    rawMsgDataSZ = pd.read_pickle(os.path.join(readPath, 'mdLog_msg_%s.pkl'%thisDate))
    orderDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '2'][['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'OrderQty', 'Price', 'cum_volume', "agg", "TransactTime"]].reset_index(drop=True)
    orderDataSZ['updateType'] = 0
    tradeDataSZ = pd.concat([rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'BidApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]],
                             rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]], sort=False)
    tradeDataSZ['ApplSeqNum'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), tradeDataSZ['OfferApplSeqNum'], tradeDataSZ['BidApplSeqNum'])
    tradeDataSZ['Side'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), 2, 1)
    tradeDataSZ = tradeDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]
    tradeDataSZ['updateType'] = 4
    cancelDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '4'][['SecurityID', 'BidApplSeqNum', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]].reset_index(drop=True)
    cancelDataSZ['ApplSeqNum'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, cancelDataSZ['OfferApplSeqNum'], cancelDataSZ['BidApplSeqNum'])
    cancelDataSZ['Side'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, 2, 1)
    cancelDataSZ = cancelDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradeQty', 'cum_volume', "TransactTime"]]
    cancelDataSZ['updateType'] = 3

    msgDataSZ = pd.concat([orderDataSZ, tradeDataSZ, cancelDataSZ], sort=False)
    del orderDataSZ
    del tradeDataSZ
    del cancelDataSZ
    msgDataSZ = msgDataSZ.sort_values(by=['SecurityID', 'ApplSeqNum', 'sequenceNo']).reset_index(drop=True)
    msgDataSZ['TradePrice'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradePrice'], 0)
    msgDataSZ['TradePrice'] = msgDataSZ['TradePrice'].astype('int64')
    msgDataSZ['TradeQty'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradeQty'], 0)
    msgDataSZ['TradeQty'] = msgDataSZ['TradeQty'].astype('int64')
    msgDataSZ['secid'] = msgDataSZ['SecurityID'] + 2000000
    assert(msgDataSZ['ApplSeqNum'].max() < 1e8)
    msgDataSZ['StockSeqNum'] = msgDataSZ['SecurityID']*1e8 + msgDataSZ['ApplSeqNum']
    msgDataSZ['date'] = int(thisDate) 
    print('finish market orders')


    # 2. orderLog
    infoData = orderLog[(orderLog['date'] == int(thisDate)) & (orderLog["isMsg"] == 1) 
                        & (orderLog['updateType'].isin([0, 3, 4]))].reset_index(drop=True)
    del orderLog
    infoData['Price'] = infoData['orderPrice'].apply(lambda x: round(x*100, 0))
    infoData['Price'] = infoData['Price'].astype('int64')*100
    infoData['OrderQty'] = infoData['absOrderSize']
    infoData['Side'] = np.where(infoData['orderDirection1'] == 1, 1, 2)
    infoData['TradePrice'] = np.where(infoData['updateType'] == 4, round(infoData['tradePrice']*100, 0), 0)
    infoData['TradePrice'] = infoData['TradePrice'].astype('int64')*100
    statusInfo = infoData.groupby(['order'])['updateType'].apply(lambda x: tuple(x)).reset_index()
    statusInfo.columns = ['order', 'statusLs']
    tradePriceInfo = infoData.groupby(['order'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['order', 'TradePriceLs']
    tradeQtyInfo = infoData.groupby(['order'])['absFilledThisUpdate'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['order', 'TradeQtyLs']
    infoData = infoData[infoData['updateType'] == 0]
    infoData = pd.merge(infoData, statusInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradePriceInfo, how='left', on=['order'], validate='one_to_one')
    infoData = pd.merge(infoData, tradeQtyInfo, how='left', on=['order'], validate='one_to_one')
    infoData['brokerNum'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])['colo_account'].transform('count')
    display(infoData[infoData['brokerNum'] >= 2].groupby(['colo', 'accCode'])['date'].count())
    display('%.2f%%'%(infoData[infoData['brokerNum'] >= 2].shape[0] / infoData.shape[0]*100))
    display(infoData[infoData['brokerNum'] >= 2].shape[0])
    display(infoData.shape[0])
    infoData = infoData[infoData['brokerNum'] == 1]
    infoData = infoData[['date', 'secid', 'vai', 'ars', "mrstaat", "mrstauc", 'exchange', 'group', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum', 'order', 'colo', 'accCode']]   

    print('finish our orders')

    # 3. find the position of our orders
    checkLog = msgDataSZ[msgDataSZ['updateType'] == 0]
    checkLog = pd.merge(checkLog, infoData.drop_duplicates(subset=['secid', 'ApplSeqNum'])[['secid', 'ApplSeqNum', 'ars']], 
                    on=['secid', 'ApplSeqNum'], how='left')
    checkLog['ApplSeqNum'] = np.where(~checkLog['ars'].isnull(), checkLog['ApplSeqNum'], checkLog['ars'])
    checkLog['start_time'] = np.where(~checkLog['ars'].isnull(), checkLog['clockAtArrival'], checkLog['ars'])
    checkLog.drop(["ars"],axis=1,inplace=True)
    checkLog['ApplSeqNum'] = checkLog.groupby(['secid'])['ApplSeqNum'].ffill()
    checkLog['start_time'] = checkLog.groupby(['secid'])['start_time'].ffill()
    checkLog['end_time'] = checkLog['start_time'] + 100*1e3
    checkLog = checkLog[(~checkLog['ApplSeqNum'].isnull()) & (checkLog['clockAtArrival'] > checkLog['start_time']) & 
                       (checkLog['clockAtArrival'] < checkLog['end_time'])]
    msgDataSZ = msgDataSZ[msgDataSZ['StockSeqNum'].isin(checkLog['StockSeqNum'].values)]
    print('finish get the interval')

    statusInfo = msgDataSZ.groupby(['StockSeqNum'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    statusInfo.columns = ['StockSeqNum', 'statusLs']
    tradePriceInfo = msgDataSZ.groupby(['StockSeqNum'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
    tradePriceInfo.columns = ['StockSeqNum', 'TradePriceLs']
    tradeQtyInfo = msgDataSZ.groupby(['StockSeqNum'])['TradeQty'].apply(lambda x: tuple(x)).reset_index()
    tradeQtyInfo.columns = ['StockSeqNum', 'TradeQtyLs']
    del msgDataSZ
    checkLog = pd.merge(checkLog, statusInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradePriceInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
    checkLog = pd.merge(checkLog, tradeQtyInfo, how='left', on=['StockSeqNum'], validate='one_to_one')  
    del statusInfo
    del tradePriceInfo
    del tradeQtyInfo


    try:
        checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], validate='many_to_one')
    except:
        print('There are orders with same pattern but different vai, same ApplSeqNum')
        display([infoData[infoData.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)]])
        checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])
    del infoData
    savePath = 'L:\\orderLog\\result\\marketPos'
    checkLog.reset_index(drop=True).to_pickle(os.path.join(savePath, 'SZspeed_%s.pkl'%thisDate))
    del checkLog 
    print('finished')

        
        
from twisted.internet import task, reactor
import schedule


def sleeptime(hour,min,sec):
    return hour*3600 + min*60 + sec


import tarfile
import os
import numpy as np
import glob
import shutil
import datetime
def un_gz(file_name, dirs):
    t = tarfile.open(file_name)
    dirs = dirs + '\\' + os.path.basename(file_name).split('.')[0]
    os.mkdir(dirs)
    t.extractall(path=dirs)
    da_te = os.path.basename(file_name).split('_')[1]
    readPath = dirs + '\\md***' + da_te + '***'
    dataPathLs = np.array(glob.glob(readPath))
    readPath = dirs + '\\***'
    dataPathLs1 = np.array(glob.glob(readPath))
    for i in list(set(dataPathLs1) - set(dataPathLs)):
        if os.path.exists(i):
            if os.path.isfile(i):
                os.remove(i)
            if os.path.isdir(i):
                shutil.rmtree(i) 
    for i in dataPathLs:
        if os.path.getsize(i) < 1000000000:
            os.remove(i)
    print('finish ' + da_te)
    

class Test(object):
    def __init__(self):
        self.status = True        
    def test(self):
        while self.status == True:
            try:
                print(datetime.datetime.now())
                date = (datetime.datetime.today()).strftime('%Y%m%d')
                readPath = '\\\\192.168.10.28\\equityTradeLogs'
                dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
                dateLs = np.array([int(os.path.basename(i).split('_')[1].split('.')[0]) for i in dataPathLs])
                assert(np.max(dateLs) == int(date))
#                 readPath = '\\\\192.168.10.34\\random_backup\\Kevin_zhenyu\\rawData\\***'
#                 dataPathLs = np.array(glob.glob(readPath))
#                 dateLs = np.array([int(os.path.basename(i).split('_')[1]) for i in dataPathLs])
#                 assert(np.max(dateLs) == int(date))
                readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + date + '\\***zs_92_01***'
                dataPathLs = np.array(glob.glob(readPath))
                assert(len(dataPathLs) == 1)
                un_gz(dataPathLs[0], '\\\\192.168.10.34\\random_backup\\Kevin_zhenyu\\rawData')
                print('We start to generate data now')  
                download_data()
                self.status = False
            except:
                print('still wait for data coming')
                second = sleeptime(0,5,0)
                time.sleep(second)

test1 = Test()
schedule.every().day.at("21:00").do(test1.test)
while True:
    schedule.run_pending()
    time.sleep(1)

2020-11-27 21:00:00.604791
still wait for data coming
2020-11-27 21:05:01.037309
still wait for data coming
2020-11-27 21:10:01.156316
still wait for data coming
2020-11-27 21:15:01.294805
still wait for data coming
2020-11-27 21:20:01.435141
still wait for data coming
2020-11-27 21:25:01.555215
still wait for data coming
2020-11-27 21:30:01.675671
still wait for data coming
2020-11-27 21:35:01.839680
still wait for data coming
2020-11-27 21:40:01.959731
still wait for data coming
2020-11-27 21:45:02.078798
still wait for data coming
2020-11-27 21:50:02.220207
still wait for data coming
2020-11-27 21:55:02.337800
still wait for data coming
2020-11-27 22:00:02.457190
still wait for data coming
2020-11-27 22:05:02.595754
still wait for data coming
2020-11-27 22:10:02.716726
still wait for data coming
2020-11-27 22:15:02.836429
still wait for data coming
2020-11-27 22:20:02.988926
still wait for data coming
2020-11-27 22:25:03.109625
still wait for data coming
2020-11-27 22:30:03.228978
s

2020-11-28 09:40:06.653885
still wait for data coming
2020-11-28 09:45:06.662880
still wait for data coming
2020-11-28 09:50:06.670876
still wait for data coming
2020-11-28 09:55:06.684864
still wait for data coming
2020-11-28 10:00:06.693851
still wait for data coming
2020-11-28 10:05:06.701853
still wait for data coming
2020-11-28 10:10:06.710849
still wait for data coming
2020-11-28 10:15:06.724820
still wait for data coming
2020-11-28 10:20:06.732822
still wait for data coming
2020-11-28 10:25:06.740810
still wait for data coming
2020-11-28 10:30:06.749270
still wait for data coming
2020-11-28 10:35:06.763137
still wait for data coming
2020-11-28 10:40:06.771239
still wait for data coming
2020-11-28 10:45:06.780233
still wait for data coming
2020-11-28 10:50:06.788228
still wait for data coming
2020-11-28 10:55:06.802209
still wait for data coming
2020-11-28 11:00:06.810634
still wait for data coming
2020-11-28 11:05:06.819623
still wait for data coming
2020-11-28 11:10:06.827616
s

In [2]:
from twisted.internet import task, reactor
import schedule
import time
import datetime


def test():
    print(datetime.datetime.now())
    print(time.time())
    try:
        assert(time.time() < 1606460781)
    except:
        print(1)
        if reactor.running:
            reactor.stop()

def loop():
    print('new loop from here')
    timeout = 5.0
    l = task.LoopingCall(test)
    l.start(timeout)
    reactor.run()

if __name__ == '__main__':
    loop()

new loop from here
2020-11-27 15:06:47.369107
1606460807.369107
1
2020-11-27 15:06:52.369689
1606460812.3696892
1
new loop from here
2020-11-27 15:07:12.379315
1606460832.3793154
1


ReactorNotRestartable: 

In [ ]:
from twisted.internet import task, reactor
import schedule
import time
import datetime

class Test(object):
    def __init__(self):
        self.l = None        
    def test(self):
        print(datetime.datetime.now())
        print(time.time())
        try:
            assert(time.time() < 1606460781)
        except:
            print(1)
#             print(reactor.running)
#             self.l.stop()
            print(reactor.running)
            return()

def loop():
    print('new loop from here')
    timeout = 5.0
    Test1 = Test()
    l = task.LoopingCall(Test1.test)
    Test1.l = l
    l.start(timeout)
    reactor.run()

schedule.every(20).seconds.do(loop)
if False:
    print('here')
while True:
    schedule.run_pending()
    time.sleep(1)

new loop from here
2020-11-27 15:20:01.268657
1606461601.268657
1
False
2020-11-27 15:20:06.269057
1606461606.2690573
1
True
2020-11-27 15:20:11.269439
1606461611.2694392
1
True
2020-11-27 15:20:16.269151
1606461616.2691512
1
True
2020-11-27 15:20:21.269245
1606461621.2692451
1
True
2020-11-27 15:20:26.269235
1606461626.2692351
1
True
2020-11-27 15:20:31.269356
1606461631.269356
1
True
2020-11-27 15:20:36.269400
1606461636.2694006
1
True
